In [1]:
import os
import pandas as pd

import geopandas as gpd 

In [2]:
## shape with states

os.listdir("states_pol")
#
# Path to the shapefile
shapefile_path = "states_pol/cb_2018_us_state_500k.shp"
# Read the shapefile
states_df = gpd.read_file(shapefile_path)
#
len(states_df) 

56

In [3]:
# shape with zip codes

# Path to the shapefile
shapefile_path = "zip_codes_pol/tl_2019_us_zcta510.shp"
# Read the shapefile
zips_df = gpd.read_file(shapefile_path)

In [4]:
zips_df.dtypes

ZCTA5CE10       object
GEOID10         object
CLASSFP10       object
MTFCC10         object
FUNCSTAT10      object
ALAND10          int64
AWATER10         int64
INTPTLAT10      object
INTPTLON10      object
geometry      geometry
dtype: object

In [5]:
zips_df.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327..."
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.08698 41.53780, -83.08256 41.537..."
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"MULTIPOLYGON (((-82.83558 41.71082, -82.83515 ..."
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650 41.25371, -83.48382 41.253..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.22229 41.53102, -83.22228 41.532..."


In [6]:
zips_df = zips_df.rename(columns={'ZCTA5CE10': 'zip_code'})

In [7]:
states_df.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [8]:
states_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


In [9]:
states_df = states_df.rename(columns={'NAME': 'state_name'})
states_df['state_name'].value_counts()

state_name
Mississippi                                     1
North Carolina                                  1
Vermont                                         1
Montana                                         1
Iowa                                            1
South Carolina                                  1
New Hampshire                                   1
Arizona                                         1
District of Columbia                            1
American Samoa                                  1
United States Virgin Islands                    1
New Jersey                                      1
Maryland                                        1
Maine                                           1
Hawaii                                          1
Delaware                                        1
Guam                                            1
Commonwealth of the Northern Mariana Islands    1
Rhode Island                                    1
Kentucky                               

In [10]:
states_df = states_df.rename(columns={'STUSPS': 'state_code'})
states_df['state_code'].value_counts()

state_code
MS    1
NC    1
VT    1
MT    1
IA    1
SC    1
NH    1
AZ    1
DC    1
AS    1
VI    1
NJ    1
MD    1
ME    1
HI    1
DE    1
GU    1
MP    1
RI    1
KY    1
OH    1
WI    1
OR    1
ND    1
AR    1
IN    1
MN    1
IL    1
NV    1
AK    1
PR    1
OK    1
VA    1
WV    1
LA    1
MI    1
MA    1
ID    1
FL    1
NE    1
WA    1
NM    1
SD    1
KS    1
TX    1
CA    1
AL    1
GA    1
PA    1
MO    1
CO    1
UT    1
TN    1
WY    1
NY    1
CT    1
Name: count, dtype: int64

In [11]:
# Join both geo dataframes 

# Ensure both GeoDataFrames use the same CRS
if states_df.crs != zips_df.crs:
    zips_df = zips_df.to_crs(states_df.crs)

# Spatial join ZIP codes to states
zips_with_states = gpd.sjoin(zips_df, states_df, how="left", predicate='intersects')

In [12]:
zips_with_states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 36797 entries, 0 to 33143
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   zip_code     36797 non-null  object  
 1   GEOID10      36797 non-null  object  
 2   CLASSFP10    36797 non-null  object  
 3   MTFCC10      36797 non-null  object  
 4   FUNCSTAT10   36797 non-null  object  
 5   ALAND10      36797 non-null  int64   
 6   AWATER10     36797 non-null  int64   
 7   INTPTLAT10   36797 non-null  object  
 8   INTPTLON10   36797 non-null  object  
 9   geometry     36797 non-null  geometry
 10  index_right  36797 non-null  int64   
 11  STATEFP      36797 non-null  object  
 12  STATENS      36797 non-null  object  
 13  AFFGEOID     36797 non-null  object  
 14  GEOID        36797 non-null  object  
 15  state_code   36797 non-null  object  
 16  state_name   36797 non-null  object  
 17  LSAD         36797 non-null  object  
 18  ALAND        36797 non-

In [13]:
# Filter out US territories 
zips_with_states=zips_with_states[ [_ not in ["Puerto Rico", "Guam", "United States Virgin Islands", "Commonwealth of the Northern Mariana Islands", "American Samoa"] for _ in zips_with_states['state_name']] ]
zips_with_states['state_name'].value_counts()

state_name
Texas                   2030
Pennsylvania            1944
New York                1882
California              1808
Illinois                1522
Ohio                    1321
Missouri                1169
Iowa                    1043
Virginia                1035
Michigan                1024
Florida                 1016
Minnesota                975
North Carolina           903
Kentucky                 899
Indiana                  880
Wisconsin                851
Georgia                  834
West Virginia            832
Kansas                   805
Oklahoma                 759
Tennessee                742
Alabama                  730
Nebraska                 691
Arkansas                 685
New Jersey               639
Washington               636
Massachusetts            592
Colorado                 578
Louisiana                568
Maryland                 564
Mississippi              493
South Carolina           483
Oregon                   475
Maine                    451
Sou

In [14]:
zips_with_states.head()

,zip_code,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,index_right,STATEFP,STATENS,AFFGEOID,GEOID,state_code,state_name,LSAD,ALAND,AWATER
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327...",48,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.08698 41.53780, -83.08256 41.537...",48,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"MULTIPOLYGON (((-82.83558 41.71082, -82.83515 ...",48,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650 41.25371, -83.48382 41.253...",48,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.22229 41.53102, -83.22228 41.532...",48,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702


In [17]:
zips_with_states = zips_with_states[["zip_code","state_name", "state_code","geometry"]]

In [18]:
# save DataFrane 
zips_with_states.to_csv("data/zip_codes_per_state.csv", index=False)